### Preparing HDFS
Using magic

Create input folder on HDFS if not exists

Copy from data from local

<span style="color:red"><em style=font-size:40px;>!</em>The following cells are basically a copy and paste of our assessment 1 submission

In [46]:
!pwd
! hadoop fs -rm -R /tmp/rs_input
! hadoop fs -mkdir -p  /tmp/rs_input
! hadoop fs -put   -p  ./../data-raw/Melbourne_housing_FULL.csv             /tmp/rs_input/raw.csv
! hadoop fs -ls        /tmp/rs_input/

/home/big-data-realestate-master/scripts

20/06/07 03:33:08 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.


Deleted /tmp/rs_input


Found 1 items


-rwxrwxrwx   1 1000 staff    5018236 2020-06-04 09:20 /tmp/rs_input/raw.csv




In [47]:
//load raw into df
val df_raw = spark
    .read
    .format("csv")
    .option("header", "true")
    .load("hdfs://localhost:9000/tmp/rs_input/raw.csv")

df_raw: org.apache.spark.sql.DataFrame = [Suburb: string, Address: string ... 19 more fields]


In [48]:
//only select columns we need now
var df_working= df_raw.select("Price",
                          "Method",
                          "Type",
                          "Distance",
                          "Rooms",
                          "Bathroom",
                          "Car",
                          "Landsize",
                          "Lattitude",
                          "Longtitude",    
                          "Suburb",
                          "Address",
                          "Date")


//add meaningful to column names
df_working = df_working.withColumnRenamed("Method","MethodOfSale")
    .withColumnRenamed("Distance","DistanceFromCBD")
    .withColumnRenamed("Type","PropertyType")
    .withColumnRenamed("Lattitude","Latitude")

df_working: org.apache.spark.sql.DataFrame = [Price: string, MethodOfSale: string ... 11 more fields]
df_working: org.apache.spark.sql.DataFrame = [Price: string, MethodOfSale: string ... 11 more fields]


In [49]:
//when profiling there are a number of columns with a "#N/A" which need to be removed
df_working = df_working.filter($"DistanceFromCBD" =!= "#N/A")

df_working: org.apache.spark.sql.DataFrame = [Price: string, MethodOfSale: string ... 11 more fields]


In [50]:
//refactored to remove the for column loop
df_working = df_working.withColumn("Price",col("Price").cast("Double"))
    .withColumn("Rooms",col("Rooms").cast("Int"))
    .withColumn("DistanceFromCBD",col("DistanceFromCBD").cast("Double"))
    .withColumn("Bathroom",col("Bathroom").cast("Int"))
    .withColumn("Car",col("Car").cast("Int"))
    .withColumn("Landsize",col("Landsize").cast("Double"))
    .withColumn("Latitude",col("Latitude").cast("Double"))
    .withColumn("Longtitude",col("Longtitude").cast("Double"))
    

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: string ... 11 more fields]


In [51]:
//convert categorically values to ints
//make sure the categorical type is upper
df_working = df_working.withColumn("PropertyType", upper(col("PropertyType")))

df_working = df_working.withColumn("PropertyType",
when(col("PropertyType") === "H", "1")
.when(col("PropertyType") === "U", "2")
.when(col("PropertyType") ==="T", "3")
.otherwise("0"))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: string ... 11 more fields]
df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: string ... 11 more fields]


In [52]:
//convert categorically values to ints
//make sure the categorical type is upper
df_working = df_working.withColumn("MethodOfSale", upper(col("MethodOfSale")))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: string ... 11 more fields]


In [53]:
df_working = df_working.withColumn("MethodOfSale",
when(col("MethodOfSale") === "S", "1")
.when(col("MethodOfSale") === "SP", "2")
.when(col("MethodOfSale") === "PI", "3")
.when(col("MethodOfSale") === "PN", "4")
.when(col("MethodOfSale") === "SN", "5")
.when(col("MethodOfSale") === "VB", "6")
.when(col("MethodOfSale") === "W", "7")
.when(col("MethodOfSale") === "SA", "8")
.when(col("MethodOfSale") === "SS", "9")                                
.otherwise("0"))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: string ... 11 more fields]


In [54]:
//cast categorical values to Ints
//Not strictly needed with the one hot encoding later
df_working = df_working.withColumn("PropertyType",col("PropertyType").cast("Int"))
    .withColumn("MethodOfSale",col("MethodOfSale").cast("Int"))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [55]:
// make first letter of suburb upper case
df_working= df_working.withColumn("Suburb", initcap(col("Suburb")))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [56]:
//split address on Street name and Suffix
df_working = df_working.withColumn("StreetName",split(col("Address")," ").getItem(1)).
    withColumn("StreetSuffix",split(col("Address")," ").getItem(2)).drop("Address")

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 12 more fields]


In [57]:
//fix "The Parade, The *** adddresses"
df_working = df_working.withColumn("StreetName",
when(col("StreetName").like("The"), concat(lit("The "),col("StreetSuffix")))
.otherwise(col("StreetName")))

//remove the street suffix
df_working = df_working.withColumn("StreetSuffix",
when(col("StreetName").contains("The"), lit(""))
.otherwise(col("StreetSuffix")))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 12 more fields]
df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 12 more fields]


In [58]:
//Rebuild street name from cleaned tokens
//this is approach was due to the legacy code we already had tokenizing the address so i just rejoined them
//one could write a regex replace to remove the street numbers unit numbers etc. bu i know these columns are clean
df_working = df_working.withColumn("StreetName", concat(col("StreetName"), lit(" "),col("StreetSuffix"))).drop("StreetSuffix")

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [59]:
// make first letter of Street upper case
df_working= df_working.withColumn("StreetName", initcap(col("StreetName")))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [60]:
//drop all properties with land area less than 12 sqm 
df_working = df_working.filter(!($"Landsize"<12))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [61]:
//drop rows where type = h and landsize < 50 sqm
df_working = df_working.filter(!($"Landsize"<50 && $"PropertyType"===1))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [62]:
val df_not_null = df_working.na.drop()

df_not_null: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [63]:
df_not_null.printSchema()

root
 |-- Price: double (nullable = true)
 |-- MethodOfSale: integer (nullable = true)
 |-- PropertyType: integer (nullable = true)
 |-- DistanceFromCBD: double (nullable = true)
 |-- Rooms: integer (nullable = true)
 |-- Bathroom: integer (nullable = true)
 |-- Car: integer (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longtitude: double (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- StreetName: string (nullable = true)



In [64]:
! hadoop fs -rm -R /tmp/output
! hadoop fs -mkdir -p /tmp/output

20/06/07 03:33:20 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.


Deleted /tmp/output




In [65]:
//this coalesce and write out isn't necessary if doing e2e
val df_output = df_not_null.coalesce(1)
   .write
   .format("csv")
   .option("header","true")
   .mode("overwrite").option("sep",",")
   .save("hdfs://localhost:9000/tmp/output")

df_output: Unit = ()


In [66]:
! hadoop fs -mkdir -p /tmp/output
! rm ./../data-clean/cleanMelbourneData.csv
! hadoop fs -copyToLocal /tmp/output/\*.csv ./../data-clean/cleanMelbourneData.csv

##  <span style="color:red"><em style=font-size:40px;>!</em>Assessment 2 Modelling etc. Starts from here


## Import Data
Here you have the option to just start from the outputed file from the wrangling if that has been ran before. 
Or can go e2e

I think ideally we would want to move from output to a new input folder somewhere on HDFS

In [67]:
//use the set wrangled from above
var df_clean = df_not_null

df_clean: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


### Load from saved local file

ctrl + / or cmd + / should can comment / uncomment selections

In [68]:
// ! hadoop fs -mkdir -p  /tmp/output
// ! hadoop fs -put   -p  ./../data-clean/cleanMelbourneData.csv  /tmp/output  

In [69]:
// // Load Clean Dataset into a DataFrame from HDFS after wrangling is completed
// var df_clean = spark
//     .read
//     .format("csv")
//     .option("header", "true")
//     .load("hdfs://localhost:9000/tmp/input/*.csv")
// df_clean.cache()

// //when we use the output from the wrangle 
// //this can be removed as types should be fine
// df_clean = df_clean.withColumn("Price",col("Price").cast("Double"))
//     .withColumn("Rooms",col("Rooms").cast("Int"))
//     .withColumn("DistanceFromCBD",col("DistanceFromCBD").cast("Double"))
//     .withColumn("MethodOfSale",col("MethodOfSale").cast("Int"))
//     .withColumn("PropertyType",col("PropertyType").cast("Int"))
//     .withColumn("Bathroom",col("Bathroom").cast("Int"))
//     .withColumn("Car",col("Car").cast("Int"))
//     .withColumn("Landsize",col("Landsize").cast("Double"))
//     .withColumn("Latitude",col("Latitude").cast("Double"))
//     .withColumn("Longtitude",col("Longtitude").cast("Double"))


In [77]:
//debug check file load
df_clean.cache()
df_clean.printSchema()
df_clean.count()

root
 |-- Price: double (nullable = true)
 |-- MethodOfSale: integer (nullable = true)
 |-- PropertyType: integer (nullable = true)
 |-- DistanceFromCBD: double (nullable = true)
 |-- Rooms: integer (nullable = true)
 |-- Bathroom: integer (nullable = true)
 |-- Car: integer (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longtitude: double (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Date: long (nullable = true)
 |-- StreetName: string (nullable = true)
 |-- loggedPrice: double (nullable = true)



res21: Long = 15728


 ### Check Spark Parameters

In [71]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.{SparkConf,SparkContext}

val cs = spark.sparkContext.getConf
sc.getConf.getAll.foreach { println }

(spark.driver.port,46549)
(spark.driver.host,a56a95a92af2)
(spark.rdd.compress,True)
(spark.repl.class.outputDir,/tmp/tmpmtmmykdx)
(spark.serializer.objectStreamReset,100)
(spark.master,local[*])
(spark.executor.id,driver)
(spark.submit.deployMode,client)
(spark.repl.class.uri,spark://a56a95a92af2:46549/classes)
(spark.app.name,spylon-kernel)
(spark.ui.showConsoleProgress,true)
(spark.app.id,local-1591268382740)


import org.apache.spark.sql.SparkSession
import org.apache.spark.{SparkConf, SparkContext}
cs: org.apache.spark.SparkConf = org.apache.spark.SparkConf@2bc18d0c


### Construct vectors from attributes
#### Transform Sale Date into a numeric value

In [72]:
//does this don't do much for Pat's LR model
//so i removed it, but i have left it here incase others used it
df_clean = df_clean.withColumn("Date",unix_timestamp($"Date", "dd/mm/yyyy"))

df_clean: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [73]:
import org.apache.spark.ml.feature.{FeatureHasher,OneHotEncoder,VectorAssembler}

//PK i have refactored the separte instances of hashers, encoders and assemblers to just using the feature hasher
//this will handle all the categorical items and and one hot encoding in single call

//set the feature names
val featureColumnNames= Array("MethodOfSale",
            "PropertyType",
            "DistanceFromCBD",
            "Rooms",
            "Bathroom",
            "Car",
            "Landsize",
            "Latitude",
            "Longtitude",
            "Suburb",
            "Date",
            "StreetName")


//set the categorical names
val categoricalFeatureColumnNames= Array("MethodOfSale",
            "PropertyType",
            "Suburb",
            "StreetName")

//define hasher
val hasher = new FeatureHasher()
  .setInputCols(featureColumnNames)
  .setCategoricalCols(categoricalFeatureColumnNames)
  .setOutputCol("hashedFeatures")


import org.apache.spark.ml.feature.{FeatureHasher, OneHotEncoder, VectorAssembler}
featureColumnNames: Array[String] = Array(MethodOfSale, PropertyType, DistanceFromCBD, Rooms, Bathroom, Car, Landsize, Latitude, Longtitude, Suburb, Date, StreetName)
categoricalFeatureColumnNames: Array[String] = Array(MethodOfSale, PropertyType, Suburb, StreetName)
hasher: org.apache.spark.ml.feature.FeatureHasher = featureHasher_ad6b5228e374


### Apply Log transformation to price

In [75]:
df_clean = df_clean.withColumn("loggedPrice", log($"Price"))

df_clean: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 12 more fields]


In [110]:
df_clean.select("loggedPrice").show(4)

+------------------+
|       loggedPrice|
+------------------+
|14.207552645740298|
|13.849911984681606|
|14.197365800433305|
|13.652991628466498|
+------------------+
only showing top 4 rows



In [114]:
//check the antilog - introduces a slight rounding error
//could fix in future
df_clean.withColumn("PriceAntilog", exp((col("loggedPrice")))).select("Price","PriceAntilog").show(10)

+---------+------------------+
|    Price|      PriceAntilog|
+---------+------------------+
|1480000.0|1480000.0000000007|
|1035000.0|1034999.9999999997|
|1465000.0|1465000.0000000005|
| 850000.0| 849999.9999999994|
|1600000.0| 1600000.000000001|
| 941000.0| 940999.9999999994|
|1876000.0|1876000.0000000012|
|1636000.0| 1635999.999999999|
|1097000.0|1096999.9999999995|
|1350000.0|1350000.0000000002|
+---------+------------------+
only showing top 10 rows



### Split Data into a Training and a Testing Set

In [78]:
//define function to take a data frame and set training and test samples
//stratafied sample on propertype type
//original random sample may have been skewing the model
//used propertype as the prices vary wildly based on the type 

//idea is split into training and testing - then stratify the training set so its balanced.

//https://maet3608.github.io/nuts-ml/tutorial/split_stratify.html#id1

import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._


def train_test_split(data: DataFrame) = {
    
    //split into 80% 20%
    var Array(train, test) = data.randomSplit(Array(0.8, 0.2), seed = 30)
    
    //Stratified sampler
    //want to over sample the other propertyTypes
    val fractions = Map(1 -> 0.8,2 ->0.9, 3 -> 0.9)

     train = train.stat.sampleBy("PropertyType",fractions,36L)
    
    (train, test)
}

val (train, test) = train_test_split(df_clean)

train.cache()
test.cache()

//debug sample counts
//println(train.count())
//println(test.count())

10206
3154


import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
train_test_split: (data: org.apache.spark.sql.DataFrame)(org.apache.spark.sql.Dataset[org.apache.spark.sql.Row], org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])
train: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Price: double, MethodOfSale: int ... 12 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Price: double, MethodOfSale: int ... 12 more fields]


### 1. Apply Linear Regression


###  Useful Specs for this section 

Feel free to move to references i guess

https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.regression.LinearRegression

https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.tuning.ParamGridBuilder

https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.feature.FeatureHasher

https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression

https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html#regression-model-evaluation

https://spark.apache.org/docs/1.6.2/api/java/org/apache/spark/sql/DataFrameStatFunctions.html#sampleBy(java.lang.String,%20java.util.Map,%20long)








In [81]:
//imports for linear regression code in one spot
//to be moved with other imports to start

import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.Pipeline


In [82]:
//function for getting execution time from start and end times
def getExecutionTime(start: Long , end : Long) = {
    val duration:Long = (end - start) / 1000
    (duration)
}

getExecutionTime: (start: Long, end: Long)Long


### Run first LR model with default Params
Assess the training set

In [83]:
//Assess the above LR
val startTimeMillis = System.currentTimeMillis()

//just give a LR a go with default settings values
val lr = new LinearRegression()
    .setLabelCol("loggedPrice")
    .setFeaturesCol("hashedFeatures")
    .setPredictionCol("Predicted loggedPrice")
    .setMaxIter(1500)

//make a dataset for testing the model and printing its summarry
val featurized = hasher.transform(train)
val lrModel = lr.fit(featurized)

// Print the coefficients and intercept for linear regression
println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")

// Summarize the model over the training set and print out some metrics
val trainingSummary = lrModel.summary

println("\n")

println(s"numIterations: ${trainingSummary.totalIterations}")
println(s"objectiveHistory: [${trainingSummary.objectiveHistory.mkString(",")}]")
println("\n")

trainingSummary.residuals.show()

println("\n")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")
println(s"r2: ${trainingSummary.r2}")
println("\n")

//print runtime
val endTimeMillis = System.currentTimeMillis()

print("Model was executed "
      + getExecutionTime(startTimeMillis,endTimeMillis))


Coefficients: (262144,[23,38,85,200,209,238,240,246,345,403,441,457,508,510,562,589,603,614,619,651,706,709,743,764,845,936,943,950,1004,1025,1031,1059,1179,1249,1272,1340,1364,1382,1408,1438,1476,1512,1603,1750,1764,1815,1874,1876,1940,2065,2079,2090,2158,2242,2284,2307,2354,2389,2398,2476,2536,2620,2630,2669,2695,2702,2760,2764,2807,2835,2933,2940,3036,3101,3161,3179,3187,3208,3276,3290,3298,3351,3403,3447,3490,3556,3640,3667,3677,3735,3846,3849,3894,3996,4080,4091,4092,4093,4127,4160,4223,4266,4278,4381,4396,4411,4447,4462,4496,4549,4566,4614,4652,4656,4688,4694,4768,4780,4788,4804,4838,4843,4878,4890,4914,4960,4970,5009,5030,5125,5147,5154,5160,5166,5196,5261,5409,5417,5442,5504,5510,5516,5540,5569,5580,5632,5646,5700,5753,5777,6144,6186,6204,6239,6349,6366,6371,6463,6484,6503,6509,6512,6637,6645,6701,6732,6756,6904,6917,6943,6947,6951,6983,6998,7068,7101,7145,7202,7210,7248,7265,7326,7329,7333,7369,7385,7393,7408,7458,7560,7564,7591,7612,7617,7634,7645,7665,7783,7785,7826,7903,791

,128028,128109,128179,128221,128260,128290,128293,128328,128337,128338,128350,128372,128373,128410,128438,128442,128453,128559,128564,128663,128705,128770,128884,128936,128970,129035,129067,129071,129102,129105,129203,129323,129336,129546,129559,129560,129656,129811,129938,129999,130078,130093,130173,130186,130286,130296,130381,130423,130438,130510,130514,130521,130568,130581,130596,130723,130785,130819,130877,130897,130951,131012,131028,131101,131104,131229,131288,131457,131491,131536,131566,131710,131722,131724,131795,131828,131850,131947,131993,131997,132007,132026,132041,132102,132185,132192,132275,132329,132335,132351,132370,132396,132437,132456,132457,132553,132586,132610,132671,132676,132776,132808,132853,132878,132882,132885,132943,132985,132988,133007,133089,133091,133112,133183,133216,133254,133358,133361,133384,133391,133444,133462,133526,133548,133605,133707,133766,133877,133897,133911,133956,134069,134076,134161,134188,134197,134348,134425,134457,134557,134575,134576,13460

301,241310,241374,241432,241480,241492,241572,241588,241611,241687,241753,241863,241889,241964,241995,242079,242089,242097,242106,242126,242127,242129,242156,242277,242282,242283,242328,242343,242367,242399,242427,242470,242508,242514,242557,242713,242819,242826,242852,242908,242946,243009,243060,243074,243105,243133,243162,243164,243169,243222,243244,243282,243380,243405,243522,243534,243644,243793,243795,243799,243810,243839,243860,243965,243976,243983,243992,244051,244056,244123,244140,244143,244154,244200,244207,244252,244328,244338,244352,244357,244389,244431,244449,244460,244468,244479,244517,244533,244561,244574,244577,244581,244583,244622,244627,244639,244642,244691,244786,244830,244831,244899,244938,244985,245160,245219,245221,245379,245454,245458,245478,245523,245600,245630,245673,245738,245777,245872,245893,245964,245997,246132,246217,246552,246561,246617,246704,246716,246730,246774,246787,246853,246897,246938,246943,247060,247108,247175,247203,247209,247577,247596,247659,24

,-0.08700853258403259,-0.09058980164025285,-0.3783305590248801,0.1753046730884125,-0.2090816201818763,-0.21181459357878052,0.0412693618013245,-0.24114480854128525,0.19761179792442285,-0.08454184654745489,0.1624472728023489,0.44864046057429446,-0.028355683049418427,-0.039220844833167394,0.1307995910612545,-0.06848689536771756,0.10627344633763962,-0.1426877065259487,-0.3583340789640323,-0.226043969085105,-0.08954569854916467,-0.2591523674855625,-0.47270330162707674,-0.11566733558353942,0.08419463224242553,-0.2121063312951603,-0.07617536032460706,-0.030300127157446115,-0.30534814839902485,-0.38148161761039484,0.1529311326310691,0.04193829450574782,-0.13579954795474078,-0.04510386656635857,0.06549303928775822,-0.03269626029869563,-0.24762832172784505,-0.1499107693943253,0.5382533526117794,-0.1259924250530228,0.24385194356251075,-0.03670747734452303,-0.3924034761254892,0.03438130101629066,-0.15181194105055432,-0.11843801606090269,-0.04320274020062721,-0.0013872713114794239,-0.33371942692531

127283659121672,-0.07302663386124106,-0.46174725330472105,0.20968721587901107,0.30869689912413734,0.07174344379644827,0.22106665581922647,-0.11263189569677025,0.08809598927100276,0.0030128792920306553,-0.059424037655974614,0.2988235643984729,0.01793461939443366,0.12299712812219141,0.02895330132158631,-0.06630603345754658,-0.11152366347612888,-0.11040532261701116,0.25072931019232625,-0.32785620999503207,-0.09237860154613231,-0.3403776601334912,0.10158786711424801,0.48265234444567817,0.035407945710077375,0.15612871549944543,-0.21029762256848472,-0.15865068045135866,-0.11483292285691067,0.04506303963796128,-0.11289228016646229,0.370875360904187,0.33215495436094095,0.1275841635283812,-0.08495080166480215,-0.2751948348549414,0.1983221052862698,-0.12008396411951694,0.15598089282070338,0.1447822117375782,0.11867725614110934,-0.24333575041492558,-0.4060123280150677,-0.007830952663622671,0.3537350235570854,-0.5653594757239558,0.33409437581419743,-0.4122803802852823,-0.05554121568003973,-0.01701

...

67691549,-0.2511359768201764,-0.13175351788690035,0.11120808355535296,-0.05966903078424248,-0.18286425045767868,-0.11326719230413221,-0.003279785497843557,-0.19040662646286727,0.06727271772244883,-0.08771227866340152,-0.26265875792124144,-0.28859415391723897,-0.022699258958524093,-0.15182094070159344,0.04377339313311515,-0.10806344932511572,-0.20597496114426966,0.20456116394936757,-0.266347457332053,0.13672766904610958,-0.14554091541029188,-0.019923282185445968,-0.16768744871842722,-0.4180169177916557,-0.04517595211610984,-0.023787564049734693,0.2311186153549356,-0.28801897637109874,-0.23369664884005284,-0.004787613005481575,-0.17530116328831977,-0.23344105945330065,0.016231769676826203,-0.17059777656814562,0.3495863170277475,0.14420409015408994,0.27674938810789057,-0.13324144510063096,-0.31378628722549823,0.08961958888414669,-0.28076345763953553,0.24051606388347488,-0.2262284007751376,-0.16854176077387414,-0.13390211537243923,0.05143051927667991,0.37687789230088725,-0.1040022805792583

437576098628,0.12413804697569354,-0.0818929855920644,-0.1892797896912907,0.1625556353301941,-0.20267365608660848,-0.058469545915934014,0.10875694159146758,0.030270696301124073,-0.0941732178007872,-0.28768330151605026,-0.2388926072833638,0.21907771342317586,0.3863237994475457,-0.16336277836010957,0.11334193549424412,-0.9418148901415846,-0.03228850542310294,-0.2991892474015056,-0.16383680776174916,-0.013942938770124877,0.0831719587780706,0.0922517911576503,0.1454559271886308,-0.5920734100316535,-0.06319515138773317,0.1267046809742366,-0.18663781452231598,-0.03528643139723609,0.14564623985318853,0.9384765085784218,-0.16084044213981932,0.025928411487684842,-0.37767127807451367,-0.2288078805358361,-0.1984965637479552,-0.24324552385610962,-0.67841637455033,-0.004151116349234155,-0.2021540982030151,0.08122649115747085,-0.028594308033258672,-0.3097503451400249,-0.0730948284850626,-0.0396778670075271,-0.08444762614227729,-0.7230039878288517,-0.020577021780263473,-0.030624062678139722,-0.2485726

55,-0.21069281413754645,0.2497639618574217,0.1509502562939048,0.42173356051295563,-0.029038717397709475,0.05306527141534149,-0.1974666580618361,0.01346950114694009,0.02464905462291933,0.17893301199306552,-0.06380167459046071,0.17431587177671456,-0.448268504738851,0.06261802388654689,-0.15673597623262528,0.03934619098961648,0.20177682283394002,-0.6856583144336094,-0.5749119779760954,-0.06079576068459661,-0.13469476416831944,-0.035485939742797304,0.0018461844256801399,-0.06680399045514855,0.11426332104070276,-0.2964048129914824,-0.40528770066135006,-0.43731559836182987,-0.017492969622669823,-0.04985164503980885,0.07969222484509447,-0.12505204197598666,-0.8433201645038286,0.08796730546543419,-0.17974824348938806,-0.13022197398270938,-0.029808997771602334,-0.5321077356261757,0.3129562013176414,0.37819549551082066,0.00784067955380486,-0.08106300834569181,-0.2353545015642212,0.08124400613425324,0.25358914632588153,-0.2082096278167391,-0.14215155056345383,-0.4113035909779816,-0.31496664966086




numIterations: 167
objectiveHistory: [0.49995100921023916,0.353878136958036,0.06984884831789119,0.06177358936747949,0.05355589432280971,0.05111131980254491,0.04937906349969907,0.04800970815985179,0.04764301974529058,0.04711475480938131,0.046825971895059715,0.04669194598692878,0.0466043638441551,0.04652030733497084,0.04647039235672933,0.046409839860361984,0.04637421615612535,0.04633446584012735,0.04630197975133111,0.04627906715426361,0.04626390553521839,0.04625253766892319,0.04623337213427334,0.046229804791198155,0.04621952223810104,0.04621594249761768,0.046208242364461895,0.04620158152936883,0.046196027229689525,0.04619235926873857,0.04618947942669538,0.04618668355838203,0.046182804031115514,0.04617786244739808,0.04617480257626565,0.046172054204695244,0.04616700162604432,0.04616352642786222,0.04615934555765052,0.04615391356016616,0.046149172953667335,0.04614452293724984,0.04613786156953567,0.046131616539649554,0.046127039197551935,0.046119359066186597,0.04611731924252514,0.046109225

startTimeMillis: Long = 1591500911936
lr: org.apache.spark.ml.regression.LinearRegression = linReg_ff7331cac34e
featurized: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 13 more fields]
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = linReg_ff7331cac34e
trainingSummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@1e749c2a
endTimeMillis: Long = 1591500922760


evaluate: (predictions: org.apache.spark.sql.DataFrame, metric: String)Unit


### Run Crossvalidator on LR model
Construct paramgrid of regParam and ElasticNet

Estimate the performance using RMSE

Get best Params to apply in final pipeline

One could add this to the pipepline but this takes the longest to run
as its assessing 4*4 combinations - so instead the learnings of this are passed to pipeline
Then this wouldn't need to run

<span style="color:red"><em style=font-size:40px;>!</em>Runtime of next cell is around 200 seconds or so, on my VM with like 10gig of RAM and 4 CPUs</span>



In [87]:
//Creates a crossvalidator on only the LR model
//Had some issues retreiving the params when using on pipeline
//and the pipeline only has one estimator in it so this was easier than traversing the stages of the pipline

val startTimeMillis = System.currentTimeMillis()

// make some featurised sets for just running the model thru the Crossvalidator
val featurized_training = hasher.transform(train)
val featurized_test = hasher.transform(test)

//set LR with 100 max iter
val lr = new LinearRegression()
    .setLabelCol("loggedPrice")
    .setFeaturesCol("hashedFeatures")
    .setPredictionCol("Predicted LoggedPrice")
    .setMaxIter(1500)


// We use a ParamGridBuilder to construct a grid of parameters to search over.
val paramGrid = new ParamGridBuilder()
  .addGrid(lr.regParam, Array(0,0.1,0.5,1))
  .addGrid(lr.elasticNetParam, Array(0,0.1,0.5,1))
  .build()

// We now treat the model as an Estimator, wrapping it in a CrossValidator instance.
// This will allow us to choose best params for the model
val cv = new CrossValidator()
  .setEstimator(lr)
  .setEvaluator(new RegressionEvaluator()
  .setLabelCol("loggedPrice")
  .setPredictionCol("Predicted LoggedPrice")
  .setMetricName("rmse"))
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(3)  
  .setParallelism(2)

// Run cross-validation, and choose the best set of parameters.
val cvModel = cv.fit(featurized_training)

// Make predictions on test documents. 
//cvModel uses the best model found.
cvModel.transform(featurized_test)
  .select("loggedPrice", "Predicted LoggedPrice")
  .show()

//print runtime
val endTimeMillis = System.currentTimeMillis()

print("Model was executed "
      + getExecutionTime(startTimeMillis,endTimeMillis))


2020-06-07 03:42:22,169 WARN  [Executor task launch worker for task 90239] storage.BlockManager (Logging.scala:logWarning(66)) - Block rdd_50396_1 already exists on this machine; not re-adding it
2020-06-07 03:42:48,872 WARN  [block-manager-slave-async-thread-pool-373] storage.BlockManager (Logging.scala:logWarning(66)) - Asked to remove block broadcast_89705_piece0, which does not exist
2020-06-07 03:43:12,526 WARN  [block-manager-slave-async-thread-pool-396] storage.BlockManager (Logging.scala:logWarning(66)) - Asked to remove block broadcast_90624, which does not exist
2020-06-07 03:43:51,733 WARN  [block-manager-slave-async-thread-pool-382] storage.BlockManager (Logging.scala:logWarning(66)) - Asked to remove block broadcast_92132_piece0, which does not exist
2020-06-07 03:43:51,736 WARN  [block-manager-slave-async-thread-pool-350] storage.BlockManager (Logging.scala:logWarning(66)) - Asked to remove block broadcast_92132, which does not exist
+------------------+------------------

startTimeMillis: Long = 1591501291224
featurized_training: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 13 more fields]
featurized_test: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 13 more fields]
lr: org.apache.spark.ml.regression.LinearRegression = linReg_be25ac5db2c9
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	linReg_be25ac5db2c9-elasticNetParam: 0.0,
	linReg_be25ac5db2c9-regParam: 0.0
}, {
	linReg_be25ac5db2c9-elasticNetParam: 0.0,
	linReg_be25ac5db2c9-regParam: 0.1
}, {
	linReg_be25ac5db2c9-elasticNetParam: 0.0,
	linReg_be25ac5db2c9-regParam: 0.5
}, {
	linReg_be25ac5db2c9-elasticNetParam: 0.0,
	linReg_be25ac5db2c9-regParam: 1.0
}, {
	linReg_be25ac5db2c9-elasticNetParam: 0.1,
	linReg_be25ac5db2c9-regParam: 0.0
...

### Extract best Parameters from CrossValidation

In [89]:
//print out the params used for the best model
val bestModel = cvModel.bestModel

//save as ParamMap to pass into pipeline
val bestParamMap = bestModel.extractParamMap

//these the bestParams determined from CV
println(bestModel.extractParamMap) 

{
	linReg_be25ac5db2c9-aggregationDepth: 2,
	linReg_be25ac5db2c9-elasticNetParam: 0.0,
	linReg_be25ac5db2c9-epsilon: 1.35,
	linReg_be25ac5db2c9-featuresCol: hashedFeatures,
	linReg_be25ac5db2c9-fitIntercept: true,
	linReg_be25ac5db2c9-labelCol: loggedPrice,
	linReg_be25ac5db2c9-loss: squaredError,
	linReg_be25ac5db2c9-maxIter: 1500,
	linReg_be25ac5db2c9-predictionCol: Predicted LoggedPrice,
	linReg_be25ac5db2c9-regParam: 0.1,
	linReg_be25ac5db2c9-solver: auto,
	linReg_be25ac5db2c9-standardization: true,
	linReg_be25ac5db2c9-tol: 1.0E-6
}


bestModel: org.apache.spark.ml.Model[_] = linReg_be25ac5db2c9
bestParamMap: org.apache.spark.ml.param.ParamMap =
{
	linReg_be25ac5db2c9-aggregationDepth: 2,
	linReg_be25ac5db2c9-elasticNetParam: 0.0,
	linReg_be25ac5db2c9-epsilon: 1.35,
	linReg_be25ac5db2c9-featuresCol: hashedFeatures,
	linReg_be25ac5db2c9-fitIntercept: true,
	linReg_be25ac5db2c9-labelCol: loggedPrice,
	linReg_be25ac5db2c9-loss: squaredError,
	linReg_be25ac5db2c9-maxIter: 1500,
	linReg_be25ac5db2c9-predictionCol: Predicted LoggedPrice,
	linReg_be25ac5db2c9-regParam: 0.1,
	linReg_be25ac5db2c9-solver: auto,
	linReg_be25ac5db2c9-standardization: true,
	linReg_be25ac5db2c9-tol: 1.0E-6
}


In [91]:
//define new LR instance for using the best params in pipeline
val bestlr = new LinearRegression()
    .setLabelCol("loggedPrice")
    .setFeaturesCol("hashedFeatures")
    .setPredictionCol("Predicted loggedPrice")

bestlr: org.apache.spark.ml.regression.LinearRegression = linReg_2b06961bfa69


### Define Pipeline 

In [92]:
// add linear regression to stages
// I removed the scaler as the performance for gain was terrible
val lrStages = Array(
    hasher,
    bestlr
)

lrStages: Array[org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable}}] = Array(featureHasher_ad6b5228e374, linReg_2b06961bfa69)


In [93]:
val startTimeMillis = System.currentTimeMillis()

//define a pipleine
val lrPipe = new Pipeline().setStages(lrStages)

// We fit our DataFrame into the pipeline to generate a model
// pass best ParamMap from cross validation
val lrModel = lrPipe.fit(train,bestParamMap)


// Make predictions using the model and the test data
// pass best ParamMap from cross validation
val predictions = lrModel.transform(test,bestParamMap)

val endTimeMillis = System.currentTimeMillis()

print("Pipeline was executed "
      + getExecutionTime(startTimeMillis,endTimeMillis))


Pipeline was executed 6

startTimeMillis: Long = 1591501595889
lrPipe: org.apache.spark.ml.Pipeline = pipeline_d4ffabb905b8
lrModel: org.apache.spark.ml.PipelineModel = pipeline_d4ffabb905b8
predictions: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 14 more fields]
endTimeMillis: Long = 1591501602534


In [94]:
def getExecutionTime(start: Long , end : Long) = {
    val duration:Long = (end - start) / 1000
    (duration)
}

getExecutionTime: (start: Long, end: Long)Long


### Undo the Log transformation  on the prediciton

In [113]:
var Antilogpredictions = predictions.withColumn("Predicted loggedPrice",col("Predicted loggedPrice").cast("Double"))
Antilogpredictions.withColumn("Predicted Price", exp((col("Predicted loggedPrice")))).select("Predicted Price").show()

+------------------+
|   Predicted Price|
+------------------+
| 268463.7666907244|
|320000.71693956025|
| 308627.3274379664|
| 265183.5027409324|
| 465493.0205295025|
| 349470.4824271826|
| 608025.0925479425|
| 635982.2169280623|
|  526205.683120743|
| 378792.4026021724|
|  483081.433702172|
|383285.88383713306|
|329591.07451323996|
| 525048.2521342823|
|  414362.529620406|
| 602393.9704089357|
| 622135.3798117155|
| 558097.3574494704|
|305046.99368025234|
| 266446.8060679387|
+------------------+
only showing top 20 rows



Antilogpredictions: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 14 more fields]


In [102]:
Antilogpredictions = Antilogpredictions.withColumn("Predicted Price", exp((col("Predicted loggedPrice"))))

Antilogpredictions: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 15 more fields]


### Evaluation of the model and its predictions

In [103]:
//finesse the output of predicted price and price to aid visual compare
Antilogpredictions.withColumn("Predicted Price", round($"Predicted Price", 0))
    .select("Price","Predicted Price").show()

+--------+---------------+
|   Price|Predicted Price|
+--------+---------------+
|170000.0|       268464.0|
|280000.0|       320001.0|
|280500.0|       308627.0|
|283000.0|       265184.0|
|290000.0|       465493.0|
|300000.0|       349470.0|
|300000.0|       608025.0|
|305000.0|       635982.0|
|310000.0|       526206.0|
|316000.0|       378792.0|
|320000.0|       483081.0|
|320000.0|       383286.0|
|320000.0|       329591.0|
|320000.0|       525048.0|
|325000.0|       414363.0|
|335000.0|       602394.0|
|340000.0|       622135.0|
|346000.0|       558097.0|
|350000.0|       305047.0|
|350000.0|       266447.0|
+--------+---------------+
only showing top 20 rows



In [104]:
Antilogpredictions.describe().select("summary","Price","Predicted Price").show()

+-------+------------------+------------------+
|summary|             Price|   Predicted Price|
+-------+------------------+------------------+
|  count|              3154|              3154|
|   mean|1159751.4419784402|1165939.0849067552|
| stddev| 654280.4305455782| 589644.6926599448|
|    min|          170000.0| 255921.2454490169|
|    max|         6300000.0|  6814540.12837558|
+-------+------------------+------------------+



In [117]:
// define an evaluator for the cross validation
def evaluate ( predictions: DataFrame, metric: String) = {
    val eval =  new RegressionEvaluator()
       .setLabelCol("loggedPrice")
       .setPredictionCol("Predicted loggedPrice")
       .setMetricName(metric)
println("Root Mean Squared Error "+  metric.toUpperCase()
        +" on test data = " + eval.evaluate(predictions))
}

evaluate: (predictions: org.apache.spark.sql.DataFrame, metric: String)Unit


#### Regression metrics

**Mean squared error (MSE)** -- the average of squared differences between the predicted outcome and the true outcome.

**R2 coefficient** -- the proportion of variance in the outcome that our model is capable of predicting based on its features.

In [119]:
//difference between logged price and predicted logged price
evaluate(Antilogpredictions,"rmse")
evaluate(Antilogpredictions,"r2")

Root Mean Squared Error RMSE on test data = 0.25787988729724454
Root Mean Squared Error R2 on test data = 0.7274712899694629


Looks like we go down from our training set:

`RMSE: 0.15067909288639053
r2: 0.9080852142542626`

to our test set with best params:

`0.25787988729724454
0.7274712899694629`


### 2. Apply K-means Model  

This modelling of this section is based on (Sarkar, 2017)

#### Training


Pipeline Estimator

In [45]:
import org.apache.spark.ml.clustering.KMeans

val kmeans = new KMeans().setK(2).setSeed(1L)
val model = kmeans.fit(df)

<console>: 39: error: not found: value df

#### Prediction

#### Testing/Evaluation

Pipeline Model Transformer

Evaluate the quality of clustering by "elbowing" the **Within SEt Sum of Squared Errors (WSSSE)** graph.

In [ ]:
val WSSSE = model.computeCost(df)

println("WSSSE error + $WSSSE")

In [ ]:
model.clusterCenters.foreach(println)

In [ ]:
val transformed = model.transform(df)

In [ ]:
//compute the diff between the labels abd the predicated values on data set

transformed.select("prediction").groupBy("prediction")count().orderBy().show("prediction")

In [ ]:
val y1df = transformed.select($"label",$"prediction").where($"label!=prediction")

In [ ]:
y1df.count()

In [ ]:
transformed.filter("prediction = 0").show()

In [ ]:
transformed.filter("prediction = 1").show()

In [ ]:
transformed.filter("prediction = 0").describe.show()

In [ ]:
transformed.filter("prediction = 1").describe.show()

In [ ]:
println("No of mis-matche between predictions and labels ="+y1df.count() +"\nTotal no. of records "+transformed.count()+ \
         "\nCorrect predictions = "+ (1-y1df.count()).toDouble/transformed.count()+"\nMismatch "+\
          (1-y1df.count()).toDouble/transformed.count())

In [ ]:
//Feed test input records for the model to predict their cluster
val testData = spark.createDataFrame(Seq().toDF("colName","features"))
model.transform(testData).show()

### 3. Apply Random Forest Regression

**Build Random Forest model**
Specify maxDepth, maxBins, auto and seed parameters.

**maxDepth** -- Maximum depth of a tree. Increasing the depth makes the model more powerful, but deep trees take longer to train.

**maxBins** -- Maximum number of bins used for discretizing continuous features and for choosing how to split on features at each node.

**auto** -- Automatically select the number of features to consider for splits at each tree node

**seed** -- Use a random seed number , allowing to repeat the results


If the number of trees is 1, then no bootstrapping is used at all. However, if the number of trees is > 1, then the bootstrapping is accomplished. Where, the parameter featureSubsetStrategy signifies the number of features to be considered for splits at each node. The supported values of featureSubsetStrategy are "auto", "all", "sqrt", "log2" and "on third". The supported numerical values, on the other hand, are (0.0-1.0] and [1-n]. However, if featureSubsetStrategy is chosen as "auto", the algorithm chooses the best feature subset strategy automatically


If the numTrees == 1, the featureSubsetStrategy is set to be "all". However, if the numTrees > 1 (i.e., forest), featureSubsetStrategy is set to be "onethird" for regression


Moreover, if a real value "n" is in the range (0, 1.0] is set, n*number_of_features is used consequently. However, if an integer value "n" is in the range (1, the number of features) is set, only n features are used alternatively


The parameter categoricalFeaturesInfo which is a map is used for storing arbitrary of categorical features. An entry (n -> k) indicates that feature n is categorical with k categories indexed from 0: {0, 1,...,k-1}
The impurity criterion used for information gain calculation. The supported values are “gini" and “variance”. The former is the only supported value for classification. The latter is used for regression


The maxDepth is the maximum depth of the tree. (e.g., depth 0 means 1 leaf node, depth 1 means 1 internal node + 2 leaf nodes). However, the suggested value is 4 to get a better result


The maxBins signifies the maximum number of bins used for splitting the features; where the suggested value is 100 to get better results


Finally, the random seed is used for bootstrapping and choosing feature subsets to avoid the random nature of the results.

In [ ]:
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.tuning.CrossValidator
//import org.apache.spark.ml.Pipeline

val seed = 5043

val rf = new RandomForestRegressor()
  .setMaxBins(100)
  .setMaxDepth(6)
  .setNumTrees(10)
  .setFeatureSubsetStrategy("onethird")
  .setSeed(seed)
  .setLabelCol("Price")

In [ ]:
val rfPredictions = time{predictions(rf, train, test)}
rfPredictions.cache()

In [ ]:
rfPredictions.columns

In [ ]:
rfPredictions.withColumn("prediction", round($"prediction", 0)).select("Price","prediction").show()

#### Regression metrics


In [ ]:
evaluate(rfPredictions,"rmse")

In [ ]:
evaluate(rfPredictions,"r2")

#### Testing/Evaluation/ Parameter Tuning

Cross-validation
<span style="color:red">
TO DO: 
* finish implementation for Cross-validation 
* check if finish run in reasonable time
</span>

In [ ]:
import org.apache.spark.ml.regression.RandomForestRegressor

// Models hypoparameters
val numTrees = Seq(5,10,30)
val maxBins = Seq(50,100)
val maxDepth = Seq(2,3,5)
val impurity = Seq("gini","entropy","variance")
val featureSubsetStrategy = Seq("sqrt","onethird")

val rf = new RandomForestRegressor()
  .setLabelCol("Price")
  .setFeaturesCol("features")
  .setPredictionCol("prediction")


val rfParamMap = new ParamGridBuilder()
  .addGrid(rf.numTrees, numTrees)
  .addGrid(rf.maxDepth, maxDepth)
  .addGrid(rf.maxBins, maxBins)
  .addGrid(rf.featureSubsetStrategy, featureSubsetStrategy)
  .build()

val t0 = System.nanoTime()
val best_model = train_eval(rf, rfParamMap, train, test)
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0)/(1000000000) + " s")


#### Prediction

In [ ]:
rfPredictions.withColumn("prediction", round($"prediction", 0)).select("Price","prediction").show()

// this will add new columns rawPrediction, probability and prediction
val predictionDf = randomForestModel.transform(testData)
predictionDf.show(10)

#### Tuning

#### Bias vs Variance Graph of Error (validation error and training error) versus training set size. 


<span style="color:red">
TO DO: 
produce graph -- validation error and training error should converge
</span>


### References

Apache Spark (n.d.). Spark ML Programming Guide. Retrieved from https://spark.apache.org/docs/1.2.2/ml-guide.html

Gorczynski M. (2017). Introduction to machine learning with spark and mllib (dataframe API). Retrieved from https://scalac.io/scala-spark-ml-machine-learning-introduction/

Hydrospheredata (2020). Program creek. Scala Code Examples. Scaler Retrieved from https://www.programcreek.com/scala/org.apache.spark.ml.feature.StandardScaler

Jen G. (2020) FeatureHasher. Retrieved from https://george-jen.gitbook.io/data-science-and-apache-spark/featurehasher

Johnson S (2019). From sckit-learn to Spark ML. Retrieved from https://towardsdatascience.com/from-scikit-learn-to-spark-ml-f2886fb46852

Johnson S (2019). Housing Prices - Spark ML Project Retrieved from https://github.com/scottdjohnson/HousingPricePredictions/blob/master/HousingPrices-SparkML.ipynb

Masri A. (2019). FeatureTransformation. Retrieved from
https://towardsdatascience.com/apache-spark-mllib-tutorial-7aba8a1dce6e

Sarkar A. (2017). Learning Spark SQL. Implementing a Spark ML clustering model. Packt Publishing.

Scala Doc (n.d.) Retrieved from https://docs.scala-lang.org


(2019) Random Forest Classifier with Apache Spark Retireved from https://medium.com/rahasak/random-forest-classifier-with-apache-spark-c63b4a23a7cc

Wagle M.(2020) _Predicting House Prices using Machine Learning_. Retrieved from https://medium.com/@manilwagle/predicting-house-prices-using-machine-learning-cab0b82cd3f